In [1]:
#installing libraries,pandas, azure-blob storage,dotenv
#!pip install dotenv
#!pip install azure-storage-blob

In [2]:
#Importing libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [3]:
#Data Extration
#using try and catch exception
try:
    data = pd.read_csv('zipco_transaction.csv')
    print('data extracted succesfully')
except Exception as e:
    print(f"{e} error occurred:")

data extracted succesfully


In [4]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 0:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 1:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 2:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 3:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 4:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [6]:
#Data Transformation
#Dropping duplicates
data.drop_duplicates(inplace=True)

In [7]:
#Replacing missing/null values, filling numerical values with the mean/median
num_columns = data.select_dtypes(include= ['Float64','int64']).columns
for col in num_columns:
    data.fillna({col: data[col].mean()}, inplace=True)


In [8]:
#Replacing missing/null values, filling categorical/object values with unknown
cat_columns = data.select_dtypes(include= ['object']).columns
for col in cat_columns:
    data.fillna({col: 'unknown'}, inplace=True)

In [9]:
#CHANGING DATE DATATYPE
data['Date'] = pd.to_datetime(data['Date'])

In [10]:
#Slicing Tables
#creating product table
products = data[['ProductName']].copy().drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()
products.head(5)

,ProductID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni


In [11]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [12]:
#creating customers table
customers = data[['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail']].copy().drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()
customers.head(5)

,CustomerID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [13]:
#creating staff table
staff  = data[['Staff_Name','Staff_Email']].copy().drop_duplicates().reset_index(drop=True)
staff .index.name = 'StaffID'
staff  = staff .reset_index()
staff .head(5)

,StaffID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [14]:
#creating Transactions fact table
transactions = data.merge(customers,on=['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail'], how= 'left')\
                    .merge (products, on=['ProductName'],how= 'left')\
                    .merge( staff, on=['Staff_Name','Staff_Email'],how='left') \
                    [['ProductID','CustomerID','StaffID','Date','Quantity', 'UnitPrice', 'StoreLocation','PaymentType', 'PromotionApplied','Weather',\
                       'Temperature','StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min','OrderType','DayOfWeek','TotalSales']]
transactions.index.name ='TransactionID'
transactions = transactions.reset_index()
transactions.head(5)
                    

,TransactionID,ProductID,CustomerID,StaffID,Date,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,DayOfWeek,TotalSales
0,0,0,0,0,2023-01-01 00:00:00,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,Sunday,25.064608
1,1,1,1,1,2023-01-01 01:00:00,1,7.083070,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,Sunday,7.083070
2,2,2,2,2,2023-01-01 02:00:00,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,unknown,43,Phone Order,Sunday,33.680321
3,3,3,3,3,2023-01-01 03:00:00,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,Sunday,14.629647
4,4,4,4,4,2023-01-01 04:00:00,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,Sunday,7.577727


In [18]:
#Saving dataset/tables as csv
data.to_csv('newcleaned.csv',index = False)
products.to_csv('products.csv',index = False)
customers.to_csv('customers.csv',index = False)
staff.to_csv('staff.csv',index = False)
transactions.to_csv('transactions.csv',index = False)

In [19]:
#Data loading
#Setting up Azure blob connection using .env file
load_dotenv()
connect_str = os.getenv('CONNECT_STR')
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

container_name = os.getenv('CONTAINER_NAME')
container_client = blob_service_client.get_container_client(container_name)

In [20]:
#Loading files in tuples
files = [
    (data, 'rawdata/cleaned_main.csv'),
    (products, 'clean_data/products.csv'),
    (customers, 'clean_data/customers.csv'),
    (staff, 'clean_data/staff.csv'),
    (transactions, 'clean_data/transactions.csv'),
         
    ]

for file,blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite = True)
    print(f'{blob_name} uploaded to  Blob Storage succesfully')

rawdata/cleaned_main.csv uploaded to  Blob Storage succesfully
clean_data/products.csv uploaded to  Blob Storage succesfully
clean_data/customers.csv uploaded to  Blob Storage succesfully
clean_data/staff.csv uploaded to  Blob Storage succesfully
clean_data/transactions.csv uploaded to  Blob Storage succesfully


In [24]:
#Another method to load by defining a function
import io

In [22]:

#create a function to load into Azure blob storage as parquet file
def upload_df_to_blob_as_parquet(df,container_client,blob_name):
    buffer = io.BytesIO()
    df.to_parquet(buffer, index = False)
    buffer.seek(0)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(buffer,blob_type = "BlockBlob", overwrite = True)
    print(f'{blob_name} uploaded to  Blob Storage succesfully')

In [23]:
upload_df_to_blob_as_parquet(data,container_client, 'cleaneddata/data.parquet')
upload_df_to_blob_as_parquet(products,container_client, 'extracteddata/products.parquet')
upload_df_to_blob_as_parquet(customers,container_client, 'extracteddata/customers.parquet')
upload_df_to_blob_as_parquet(staff,container_client, 'extracteddata/staff.parquet')
upload_df_to_blob_as_parquet(transactions,container_client, 'extracteddata/transactions.parquet')

cleaneddata/data.parquet uploaded to  Blob Storage succesfully
extracteddata/products.parquet uploaded to  Blob Storage succesfully
extracteddata/customers.parquet uploaded to  Blob Storage succesfully
extracteddata/staff.parquet uploaded to  Blob Storage succesfully
extracteddata/transactions.parquet uploaded to  Blob Storage succesfully
